In [78]:
from constants import *
from urllib.parse import urlencode
from requests import get
from bs4 import BeautifulSoup
import os
from glob import glob
import shutil
import time
import re
from zipfile import ZipFile
from constants import forbidden_chars

def get_movie_site(title):
    search = URL + "/search?" + urlencode({"q": title})
    soup = BeautifulSoup(get(search).text, features="lxml")
    body = str(soup.find("div", class_="media-body"))
    movie_endp = re.search("href=\"(.*)\"", body)[1]
    return URL + movie_endp


def get_title_and_download_site(movie_site) -> tuple:
    soup = BeautifulSoup(get(movie_site).text, features="lxml")
    title = soup.find(class_="movie-main-title").text
    table = soup.find(class_="table other-subs")
    sub_link = re.search("\"/subtitle.*-english-yify.*?\"", str(table))[0].replace("\"", "")
    download_site = URL + sub_link
    return title, download_site


def get_donwload_link(download_site):
    soup = BeautifulSoup(get(download_site).text, features="html.parser")
    download_endp = soup.find(class_="download-subtitle").get("href")
    download_link = URL + download_endp
    return download_link


def donwload_file(url, dest):
    r = get(url)
    with open(dest, 'wb') as out:
        out.write(r.content)

In [62]:
title = "Death of Nintendo (2020)"
site = get_movie_site(title)
site

'https://yifysubtitles.org/movie-imdb/tt10642250'

In [63]:
title, download_site = get_title_and_download_site(site)
title, download_site

('Death of Nintendo (2020)',
 'https://yifysubtitles.org/subtitles/death-of-nintendo-2020-english-yify-386419')

In [68]:
download_link = get_donwload_link(download_site)